In [1]:
!wget http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
!tar -xf MovieSummaries.tar.gz

--2024-05-06 18:29:33--  http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48002242 (46M) [application/x-gzip]
Saving to: ‘MovieSummaries.tar.gz.10’

MovieSummaries.tar. 100%[===================>]  45.78M  4.43MB/s    in 11s     

2024-05-06 18:29:44 (4.26 MB/s) - ‘MovieSummaries.tar.gz.10’ saved [48002242/48002242]



In [2]:
import nltk
import pandas as pd

df2 = pd.read_csv("/content/MovieSummaries/plot_summaries.txt", delimiter = "\t",names=["id","text"])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42303 entries, 0 to 42302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      42303 non-null  int64 
 1   text    42303 non-null  object
dtypes: int64(1), object(1)
memory usage: 661.1+ KB


In [3]:
df2.head()

,id,text
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


Preprocess the Text Data:

In [4]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer


In [7]:
# Load the dataset
df2 = pd.read_csv("/content/MovieSummaries/plot_summaries.txt", delimiter="\t", names=["id", "text"])


In [ ]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

# Apply preprocessing to the 'text' column in the dataframe
df2['processed_text'] = df2['text'].apply(preprocess_text)


Create a Thesaurus:

In [ ]:
# Create a CountVectorizer to create a document-term matrix
vectorizer = CountVectorizer(binary=True)
dtm = vectorizer.fit_transform(df2['processed_text'])

# Get the vocabulary (words) from the CountVectorizer
words = vectorizer.get_feature_names_out()

# Multiply the document-term matrix by its transpose to get the co-occurrence matrix
co_occurrence_matrix = dtm.T * dtm

# Convert the co-occurrence matrix to a dense matrix
co_occurrence_matrix_dense = co_occurrence_matrix.toarray()

# Set a threshold for co-occurrence count
threshold = 10

# Create a thesaurus dictionary to store related words
thesaurus = {}

# Find words that co-occur more than the threshold and add them to the thesaurus
for i in range(len(words)):
    related_words = []
    for j in range(len(words)):
        if i != j and co_occurrence_matrix_dense[i][j] > threshold:
            related_words.append(words[j])
    if related_words:
        thesaurus[words[i]] = related_words

# Display thesaurus entries
for word, related_words in thesaurus.items():
    print(f"{word}: {', '.join(related_words)}")
